In [40]:
import math, random
import string

class Cromossomo():

    def __init__(self, tamanho):
        self.lista_modelos = self.criaListaModelos()
        self.tamanho = tamanho
        self.valor = []
        self.cobertura = 0
        self.avaliacao_cobertura = 0
        self.probabilidade_acumulada = 0
        
    

    
    def criaListaModelos(self):
        mod1 = list(range(0,10))
        mod2 = list(string.ascii_uppercase)
        
        #modelo = [[1..9],[A..Z]]
        modelo = [mod1, mod2]
        
        return modelo
        
    def getLetraRandom(self):
        i = random.randint(0,25)
        
        return self.lista_modelos[1][i]
    
    def set_valor(self,split_index, novo_valor):
        self.valor = novo_valor[0:split_index] + self.valor[split_index:self.tamanho]
    
    def inicializar(self):
        novo_valor = []
        for i in range(self.tamanho):
            if random.random() < .5: ##Se valor.randômico < 0.50, adiciona um numero aleatório
                novo_valor.append(random.randint(0,9))
            else: novo_valor.append(self.getLetraRandom())
                
        self.valor = novo_valor

    def crossover(self,outro_cromossomo, tam_filho):
        novo_cromossomo = Cromossomo(tam_filho)
        novo_cromossomo.inicializar()
        
        split_index = int(random.random() * self.tamanho) ##Gera o índice onde vai ocorrer a troca de Genes
        novo_valor = []
        if random.random() > .5:##Se a probabilidade for maior que 50%
            novo_valor = self.valor[0:split_index] + outro_cromossomo.valor[split_index:len(outro_cromossomo.valor)]
        else:
            novo_valor = outro_cromossomo.valor[0:split_index] + self.valor[split_index:len(outro_cromossomo.valor)]
        
        tamanho_pai = self.tamanho
        
        novo_cromossomo.set_valor(tamanho_pai, novo_valor)
        
        return novo_cromossomo

    def mutacao(self, chance_mutacao):
        inicio, fim = [[],[]]
        for i in range(self.tamanho):
            if random.random() < chance_mutacao:
                inicio = self.valor[0:i]
                fim = self.valor[i+1:self.tamanho]
                aux = self.valor[i]
                if type(aux) == int : aux = self.getLetraRandom() ##Coloca uma letra aleatória na posição mutada
                else: aux = random.randint(0,9) ##Coloca um número, caso contrário
                self.valor = self.concatena_valores(inicio,aux,fim)
                
    def concatena_valores(self, a ,b ,c):
        resultado = []
        for i in range(len(a)):
            resultado.append(a[i])
            
        resultado.append(b)
        
        for i in range(len(c)):
            resultado.append(c[i])
        
        return resultado
    
    def set_cobertura(self, cobertura):
        self.cobertura = cobertura / 100
    
    def avaliar_cobertura(self, soma_coberturas):##Em construção       
        if(soma_coberturas > 0):
            ##total = 100%
            total = 1
            self.avaliacao_cobertura = (self.cobertura * total) / soma_coberturas
        

    def __repr__(self):
        return "cromossomo:[%s] avaliacao[%.2f]" % (self.valor, self.avaliacao_cobertura)

In [73]:
import math, random
import string
import re

class GATest():

    def __init__(self, tamanho_populacao, geracoes):
        self.populacao = []
        self.tam_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.soma_coberturas = 0
        self.melhores = []
        
                                ##    1     2     3     4
        self.criterios_atingidos = [False,False,False,False]
    
    
    def inicializar_populacao(self, tam_cromossomo):
        for i in range(self.tam_populacao):
            self.populacao.append(Cromossomo(tam_cromossomo))
        for cromossomo in self.populacao:
            cromossomo.inicializar()
            
    def informar_Cobertura(self, valor):
        print("\nAvaliar Cromossomo: {}".format(valor))
        entrada = input("Critério e Cobertura atingida:").split(" ")
        
        criterios = entrada[0].split(",") ##tem que trabalhar com essa variavel
        taxa_cobertura = int(entrada[1])
        
        return taxa_cobertura
            
    def avaliar_populacao(self):##Em construção        
        for cromossomo in self.populacao:
            taxa_cobertura = self.informar_Cobertura(cromossomo.valor)
            cromossomo.set_cobertura(taxa_cobertura)
            self.soma_coberturas += cromossomo.cobertura
        
        soma_probabilidade = 0
        for cromossomo in self.populacao:
            cromossomo.avaliar_cobertura(self.soma_coberturas)
            soma_probabilidade += cromossomo.avaliacao_cobertura
            cromossomo.probabilidade_acumulada = soma_probabilidade
        
        return self.populacao

    def roleta(self):
        limite = (random.random() * self.soma_coberturas) % 1 ##Gera um limite baseado na avaliação da População sempre < 1
        self.populacao = sorted(self.populacao, key = lambda x: x.probabilidade_acumulada)
        
        
        if(limite > 0):
            i = 0
            for cromossomo in self.populacao:
                if(limite <= cromossomo.probabilidade_acumulada):
                    return i
                i += 1
        
        i = random.randint(0,(self.tam_populacao - 1))
            
        return i
            

    def nova_geracao(self, tam_filhos):
        nova_populacao = []
        for i in range(self.tam_populacao):
            pai1 = self.populacao[self.roleta()]
            pai2 = self.populacao[self.roleta()]
            filho = pai1.crossover(pai2, tam_filhos)
            filho.mutacao(.10)
            nova_populacao.append(filho)
        return nova_populacao
    
    def executar(self):
        populacoes = []
        tam_cromossomo = 1
        self.inicializar_populacao(tam_cromossomo)    
        populacoes.append(self.avaliar_populacao())
        
        tam_cromossomo = 2
        while(tam_cromossomo <= 20):
            print("\n*** Geracao {} ***".format(tam_cromossomo))
            self.populacao = self.nova_geracao(tam_cromossomo)
            populacoes.append(self.avaliar_populacao())
            '''for ind in self.populacao:
                print(ind)'''
            
            tam_cromossomo += 1     
        

        
if __name__ == "__main__":
    ga = GATest(4,5)     
    ga.executar()


Avaliar Cromossomo: ['C']
Critério e Cobertura atingida:0 0

Avaliar Cromossomo: ['O']
Critério e Cobertura atingida:0 0

Avaliar Cromossomo: [8]
Critério e Cobertura atingida:0 0

Avaliar Cromossomo: [0]
Critério e Cobertura atingida:0 0

*** Geracao 2 ***

Avaliar Cromossomo: ['C', 0]
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: [8, 'J']
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: [0, 3]
Critério e Cobertura atingida:0 0

Avaliar Cromossomo: [0, 0]
Critério e Cobertura atingida:0 0

*** Geracao 3 ***

Avaliar Cromossomo: ['C', 0, 7]
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: ['C', 0, 7]
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: ['C', 0, 'M']
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: ['C', 0, 0]
Critério e Cobertura atingida:1 25

*** Geracao 4 ***

Avaliar Cromossomo: ['C', 'K', 'M', 'M']
Critério e Cobertura atingida:0 0

Avaliar Cromossomo: [3, 0, 'M', 'E']
Critério e Cobertura atingida:1 25

Avaliar Cromossomo: ['C',

IndexError: list index out of range

In [39]:
soma_coberturas = 0.75 + 0.50 + 0.25 + 0.25

valor_analisado = 0.25  ##Qual porcentagem vai ser atribuida para o Cromossomo com essa cobertura

total = 1 ##  100%

(valor_analisado * total) / soma_coberturas

0.14285714285714285

In [62]:
random.randint(0,4 - 1)

0

In [51]:
1.39918662 % 1

0.39918662000000005